import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
from scipy import signal
import meegkit
from meegkit import dss
from meegkit.utils import create_line_data, unfold, fold, rms, tscov

%matplotlib qt5

In [7]:
from meegkit.utils.denoise import demean

In [8]:
raw = mne.io.read_raw_fif('pre_filtered_data_raw.fif', preload=True)

Opening raw data file pre_filtered_data_raw.fif...
    Range : 25000 ... 1583999 =     50.000 ...  3167.998 secs
Ready.
Opening raw data file /home/ijekt/Documents/cs_cog_fat/pre_filtered_data_raw-1.fif...
    Range : 1584000 ... 2135999 =   3168.000 ...  4271.998 secs
Ready.
Reading 0 ... 2110999  =      0.000 ...  4221.998 secs...


In [9]:
# extract meg channels
meg_channels = mne.pick_types(raw.info, meg=True, eeg=False)

In [10]:
# extract eog and ecg channels
eog_channels = mne.pick_types(raw.info, meg=False, eeg=False, eog=True)
ecg_channels = mne.pick_types(raw.info, meg=False, eeg=False, ecg=True)

In [11]:
# extract meg, exg, eog data
meg_data = raw.get_data(picks=meg_channels)
#eog_data = raw.get_data(picks=eog_channels)
#ecg_data = raw.get_data(picks=ecg_channels)

In [12]:
print(meg_data.shape)
#print(eog_data.shape)
#print(ecg_data.shape)

(306, 2111000)


In [16]:
x = meg_data.T
sf = raw.info['sfreq']
x = demean(x)

In [18]:
print(x.shape)

(2111000, 306)


In [19]:
# Remove eyeblink components
w = np.abs(x) > 4 * np.median(np.abs(x))
w = np.mean(w, axis=1)
c0 = np.cov(x, rowvar=False)
c1 = np.cov(np.multiply(x, w[:, np.newaxis]), rowvar=False)
todss, fromdss, pwr0, pwr1 = meegkit.dss.dss0(c0, c1)


# Plot
plt.figure(4)
plt.clf()
plt.subplot(121)
plt.plot(pwr1 / pwr0, '.-')
plt.ylabel('score')
plt.xlabel('components')
plt.title('DSS eyeblink')

# Apply DSS
z = np.dot(x, todss)

In [21]:
print(yy.shape)

(2111000, 306)


In [11]:
# Repeat to tighten estimate
w = np.abs(z[:, 0]) > 4 * np.median(np.abs(z[:, 0]))
c0 = np.cov(x, rowvar=False)
c1 = np.cov(np.multiply(x, w[:, np.newaxis]), rowvar=False)
todss, fromdss, pwr0, pwr1 = meegkit.dss.dss0(c0, c1)

# Plotting the power ratio for the second DSS
plt.subplot(122)
plt.plot(pwr1 / pwr0, '.-')
plt.ylabel('score')
plt.xlabel('components')
plt.title('DSS eyeblink 2')

# Apply the second DSS
z = np.dot(x, todss)

In [12]:
# Number of components to keep
NKEEP = 7

# Select the best components and project back to sensor space
projection_matrix = np.dot(todss[:, :NKEEP], fromdss[:NKEEP, :])
yy = np.dot(x, projection_matrix)

In [13]:
print(yy.shape)

(2111000, 306)


In [14]:
y = yy.T

In [15]:
print(y.shape)

(306, 2111000)


In [16]:
original_ch_names = raw.ch_names[:306]
original_ch_types = ['mag'] * 102 + ['grad'] * 204

In [17]:
info = mne.create_info(ch_names=original_ch_names, sfreq=raw.info['sfreq'], ch_types=original_ch_types)

In [18]:
dss_eog_raw = mne.io.RawArray(y, info)

Creating RawArray with float64 data, n_channels=306, n_times=2111000
    Range : 0 ... 2110999 =      0.000 ...  4221.998 secs
Ready.


In [19]:
dss_eog_raw.plot(duration=5, n_channels=20, scalings='auto')

Using qt as 2D backend.


<frozen importlib._bootstrap>:228: FutureWarning: mne.io.pick.channel_indices_by_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.


Channels marked as bad:
none


In [26]:
n_channels = y.shape[0]

# Generate dummy channel names
channel_names = ['MEG%03d' % i for i in range(1, n_channels + 1)]

# Create a list of channel types (all 'meg' in this case)
channel_types = ['mag'] * n_channels

# Create MNE Info object
info = mne.create_info(ch_names=channel_names, sfreq=1000, ch_types=channel_types)  # Replace 1000 with your actual sampling rate

# Transpose 'y' to have shape [n_times, n_channels] and create RawArray object
dss_raw = mne.io.RawArray(y, info)

Creating RawArray with float64 data, n_channels=306, n_times=2111000
    Range : 0 ... 2110999 =      0.000 ...  2110.999 secs
Ready.


In [27]:
dss_raw.plot(duration=5, n_channels=20, scalings='auto')

Waiting for Loading-Thread to finish... (max. 10 sec)
